In [9]:
# import the necessary packages
import numpy as np
import imutils
import cv2
import argparse

def align_images(image, template, maxFeatures=5000, keepPercent=0.15,
	debug=False):
	# convert both the input image and template to grayscale
	imageGray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
	templateGray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

    # use ORB to detect keypoints and extract (binary) local
	# invariant features
	orb = cv2.ORB_create(maxFeatures)
	(kpsA, descsA) = orb.detectAndCompute(imageGray, None)
	(kpsB, descsB) = orb.detectAndCompute(templateGray, None)

	# match the features
	method = cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING
	matcher = cv2.DescriptorMatcher_create(method)
	matches = matcher.match(descsA, descsB, None)

    # sort the matches by their distance (the smaller the distance,
	# the "more similar" the features are)
	matches = sorted(matches, key=lambda x:x.distance)

	# keep only the top matches
	keep = int(len(matches) * keepPercent)
	matches = matches[:keep]

	# check to see if we should visualize the matched keypoints
	if debug:
		matchedVis = cv2.drawMatches(image, kpsA, template, kpsB,
			matches, None)
		matchedVis = imutils.resize(matchedVis, width=1000)
		cv2.imshow("Matched Keypoints", matchedVis)
		cv2.waitKey(0)
    
    # allocate memory for the keypoints (x, y)-coordinates from the
	# top matches -- we'll use these coordinates to compute our
	# homography matrix
	ptsA = np.zeros((len(matches), 2), dtype="float")
	ptsB = np.zeros((len(matches), 2), dtype="float")

	# loop over the top matches
	for (i, m) in enumerate(matches):
		# indicate that the two keypoints in the respective images
		# map to each other
		ptsA[i] = kpsA[m.queryIdx].pt
		ptsB[i] = kpsB[m.trainIdx].pt
    
    # compute the homography matrix between the two sets of matched
	# points
	(H, mask) = cv2.findHomography(ptsA, ptsB, method=cv2.RANSAC)

	# use the homography matrix to align the images
	(h, w) = template.shape[:2]
	aligned = cv2.warpPerspective(image, H, (w, h))

	# return the aligned image
	return aligned

In [10]:
print("[INFO] loading images...")
image = cv2.imread("pcb1.jpg")
template = cv2.imread("pcb3.jpg")

[INFO] loading images...


In [12]:
# construct the argument parser and parse the arguments
# ap = argparse.ArgumentParser()
# ap.add_argument("-i", "--image", required=True,
# 	help="path to input image that we'll align to template")
# ap.add_argument("-t", "--template", required=True,
# 	help="path to input template image")
# args = vars(ap.parse_args())

# print("[INFO] loading images...")
# image = cv2.imread("pcb_fixed.jpg")
# template = cv2.imread("pcb_rotate2.jpg")

# load the input image and template from disk
# print("[INFO] loading images...")
# image = cv2.imread(args["image"])
# template = cv2.imread(args["template"])
# align the images
print("[INFO] aligning images...")
aligned = align_images(image, template, debug=True)

#resize both the aligned and template images so we can easily
# visualize them on our screen
aligned = imutils.resize(aligned, width=700)
template = imutils.resize(template, width=700)
# our first output visualization of the image alignment will be a
# side-by-side comparison of the output aligned image and the
# template
stacked = np.hstack([aligned, template])

# our second image alignment visualization will be *overlaying* the
# aligned image on the template, that way we can obtain an idea of
# how good our image alignment is
overlay = template.copy()
output = aligned.copy()
cv2.addWeighted(overlay, 0.5, output, 0.5, 0, output)
# show the two output image alignment visualizations
cv2.imshow("Image Alignment Stacked", stacked)
cv2.imshow("Image Alignment Overlay", output)
cv2.waitKey(0)

[INFO] aligning images...


-1

## Method 2


In [1]:
from __future__ import print_function
import cv2
import numpy as np
MAX_FEATURES = 5000
GOOD_MATCH_PERCENT = 0.15
import matplotlib.pyplot as plt
%matplotlib inline
import imutils

def alignImages(im1, im2):

  # Convert images to grayscale
  im1Gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
  im2Gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)

  # Detect ORB features and compute descriptors.
  orb = cv2.ORB_create(MAX_FEATURES)
  keypoints1, descriptors1 = orb.detectAndCompute(im1Gray, None)
  keypoints2, descriptors2 = orb.detectAndCompute(im2Gray, None)

  # Match features.
  matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
  matches = matcher.match(descriptors1, descriptors2, None)

  # Sort matches by score
  # matches.sort(key=lambda x: x.distance, reverse=False)
  matches = sorted(matches, key=lambda x:x.distance)
  # Remove not so good matches
  numGoodMatches = int(len(matches) * GOOD_MATCH_PERCENT)
  matches = matches[:numGoodMatches]

  # Draw top matches
  imMatches = cv2.drawMatches(im1, keypoints1, im2, keypoints2, matches, None)
  cv2.imwrite("matches.jpg", imMatches)

  # Extract location of good matches
  points1 = np.zeros((len(matches), 2), dtype=np.float32)
  points2 = np.zeros((len(matches), 2), dtype=np.float32)

  for i, match in enumerate(matches):
    points1[i, :] = keypoints1[match.queryIdx].pt
    points2[i, :] = keypoints2[match.trainIdx].pt
  # Find homography
  h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)
  # Use homography
  height, width, channels = im2.shape
  im1Reg = cv2.warpPerspective(im1, h, (width, height))
  return im1Reg, h

if __name__ == '__main__':

  # Read reference image
  refFilename = "images/std.jpg"
  # refFilename = "pcb_fixed.jpg"
  print("Reading reference image : ", refFilename)
  imReference = cv2.imread(refFilename, cv2.IMREAD_COLOR)

  # Read image to be aligned
  imFilename = "images/mod.jpg"
  # imFilename = "pcb_rotate2.jpg"
  print("Reading image to align : ", imFilename);
  im = cv2.imread(imFilename, cv2.IMREAD_COLOR)
  print("Aligning images ...")

  # Registered image will be resotred in imReg.
  # The estimated homography will be stored in h.
  imReg, h = alignImages(im, imReference)
  # Write aligned image to disk.
  outFilename = "aligned.jpg"
  print("Saving aligned image : ", outFilename);
  cv2.imwrite(outFilename, imReg)
  # plt.imshow(imReference)
  # plt.imshow(imReg)
  # Print estimated homography
  print("Estimated homography : \n",  h)


Reading reference image :  images/std.jpg
Reading image to align :  images/mod.jpg
Aligning images ...
Saving aligned image :  aligned.jpg
Estimated homography : 
 [[ 8.09655153e-01 -4.10157478e-01  2.47998185e+02]
 [ 3.74944495e-01  8.35314498e-01 -4.75874933e+02]
 [-4.54273588e-05 -2.39290165e-05  1.00000000e+00]]


In [13]:
# def new_coord(tr_coord,h):
#     h_inv = np.linalg.inv(h)
#     or_coord = np.divide(np.dot(h_inv,tr_coord),np.dot(h_inv,tr_coord)[2])
#     return(or_coord)

In [16]:
# coord = np.array([850,1078,1]).reshape(3,-1)
# print("coord\n", coord)

# trans = np.divide(np.dot(h,coord),np.dot(h,coord)[2])
# print("trans =\n",trans)

# h_inv = np.linalg.inv(h)
# print("h_inv \n",h_inv)
# # print(h_inv)

tr_coord = np.array([890,413,1]).reshape(3,-1)
# or_coord = np.divide(np.dot(h_inv,tr_coord),np.dot(h_inv,tr_coord)[2])
# print("or_coord = \n", or_coord)
or_coord = new_coord(tr_coord,h)
print("or_coord = \n", or_coord)   
# scale = trans[2]
# trans_mod = np.divide(trans,scale)
# print("trans_mod = \n",trans_mod)

or_coord = 
 [[1.01913748e+03]
 [5.76947497e+02]
 [1.00000000e+00]]


In [169]:
def new_coord(tr_coord,h):
    h_inv = np.linalg.inv(h)
    
    fin_coord=[]
    for i in range(4):
        coord = np.append(tr_coord[i],1).reshape(3,-1)
        # print(coord,"\n")
        mod_coord = np.divide(np.dot(h_inv,coord),np.dot(h_inv,coord)[2])

        mod_coord = mod_coord.astype(int)
        mod_coord = np.delete(mod_coord,2)
        mod_coord.reshape(-1,2)
        
        fin_coord.append(mod_coord)
    fin_coord = np.array(fin_coord)
    return(fin_coord)

In [170]:
# import pandas as pd
# from ast import literal_eval

# tr_csv = pd.read_csv("../s3a-export/annotations/std.jpg.csv")
# data = tr_csv['Vertices'].to_numpy()
# print(data,"\n")
# print(data.size,"\n")

# res = np.array(literal_eval(data[0]))
# print(res,"\n")

# or_coord = new_coord(res[0],h)
# print(or_coord)


In [203]:
import pandas as pd
from ast import literal_eval

tr_csv = pd.read_csv("../s3a-export/annotations/std.jpg.csv")
data = tr_csv['Vertices'].to_numpy()
# print(data,"\n")
print(tr_csv)

for i in range(data.size):
    res = np.array(literal_eval(data[i]))
    or_coord = new_coord(res[0],h)
    or_coord = or_coord.reshape(1,4,2)
    # data[i] = np.array2string(or_coord,separator=',')
    or_coord = str(or_coord).replace('  ',', ')
    data[i] = or_coord.replace('\n,',',')

# print(data)
# print(data.shape)
# data = np.array2string(data)
tr_csv.iloc[:,2] = data
print(tr_csv)
tr_csv.to_csv(r'../s3a-export/annotations/mod.jpg.csv',index=False)

   Instance ID Image File                                           Vertices
0            0    std.jpg  [[[890, 413], [890, 439], [941, 439], [941, 41...
1            1    std.jpg  [[[1021, 409], [1021, 434], [1074, 434], [1074...
2            2    std.jpg  [[[1331, 608], [1331, 657], [1355, 657], [1355...
   Instance ID Image File                                           Vertices
0            0    std.jpg  [[[1019, 576], [1030, 600], [1076, 579], [1065...
1            1    std.jpg  [[[1136, 517], [1146, 540], [1194, 518], [1184...
2            2    std.jpg  [[[1491, 568], [1510, 612], [1531, 602], [1512...


In [250]:
# 1005,410 // -97,-91
# 1122,318 // -100,-100
# dx = -3/117
# dy = 9/92

def median(coord):
    x=0
    y=0
    for i in range(4):
        x+=coord[i][0]
        y+=coord[i][1]
    x=x/4
    y=y/4
    median_coord = np.array([x,y])
    print(median_coord)
    return(median_coord)


dx = -3/117
dy = 9/92
CNC_ratio=np.array([dx,dy])

mod_csv = pd.read_csv("../s3a-export/annotations/mod.jpg.csv")
data = mod_csv['Vertices'].to_numpy()

mod_img = cv2.imread("images/mod.jpg")
# print(mod_img.shape)
centre_x = mod_img.shape[0]/2
centre_y = mod_img.shape[1]/2
center = np.array([centre_x,centre_y])
print("Center = ",center)

medians=[]
center_diffs=[]
CNC_movements=[]

for i in range(data.size):
    res = np.array(literal_eval(data[i]))
    # print(res[0])
    median_coord = median(res[0])
    medians.append(median_coord)

    pixel_diff = np.subtract(median_coord,center)
    center_diffs.append(pixel_diff)

    CNC_movements.append(np.multiply(pixel_diff,CNC_ratio))


mod_csv['Medians'] = medians
mod_csv['Center_diffs'] = center_diffs
mod_csv['CNC_movements'] = CNC_movements

print(mod_csv)
mod_csv.to_csv(r'../s3a-export/annotations/mod_extra.jpg.csv',index=False)

Center =  [540. 960.]
[1047.5  577.5]
[1165.   517.5]
[1511.  585.]
   Instance ID Image File                                           Vertices  \
0            0    mod.jpg  [[[1019, 576], [1030, 600], [1076, 579], [1065...   
1            1    mod.jpg  [[[1136, 517], [1146, 540], [1194, 518], [1184...   
2            2    mod.jpg  [[[1491, 568], [1510, 612], [1531, 602], [1512...   

           Medians     Center_diffs  \
0  [1047.5, 577.5]  [507.5, -382.5]   
1  [1165.0, 517.5]  [625.0, -442.5]   
2  [1511.0, 585.0]  [971.0, -375.0]   

                                CNC_movements  
0   [-13.012820512820513, -37.41847826086957]  
1  [-16.025641025641026, -43.288043478260875]  
2  [-24.897435897435898, -36.684782608695656]  


In [3]:
aligned = imutils.resize(imReg, width=700)
template = imutils.resize(imReference, width=700)
# our first output visualization of the image alignment will be a
# side-by-side comparison of the output aligned image and the
# template
stacked = np.hstack([aligned, template])

# our second image alignment visualization will be *overlaying* the
# aligned image on the template, that way we can obtain an idea of
# how good our image alignment is
overlay = template.copy()
output = aligned.copy()
cv2.addWeighted(overlay, 0.5, output, 0.5, 0, output)
# show the two output image alignment visualizations
cv2.imshow("Image Alignment Stacked", stacked)
cv2.imshow("Image Alignment Overlay", output)
cv2.waitKey(0)

-1